# 🧠 Continual Learning Experiments on Time-Series Data

In this notebook, we conduct a series of experiments to evaluate **incremental learning performance** on time-series models.  
The goal is to understand how different **continual learning strategies**, **architectures**, and **normalization methods** affect model stability and knowledge retention over sequential updates.

We use the **UCI Human Activity Recognition (HAR)** dataset as our benchmark and the **TSCIL (Time-Series Continual Incremental Learning)** framework to manage training streams and agents.

Specifically, we will:
- Establish a **Sequential Fine-Tuning (SFT)** baseline to measure catastrophic forgetting.  
- Introduce **Experience Replay (ER)** as a memory-based continual learning method.  
- Compare **normalization strategies** (LayerNorm vs BatchNorm).  
- Evaluate **regularization-based methods** such as **EWC**.  
- Explore the effect of using different **encoders** (CNN vs Transformer).  

We will also track per-task and per-epoch metrics, and store results for comparison in tables and plots.

---

### Step 1: Environment Setup
In this section, we’ll import all necessary libraries for running the experiments, managing results, and visualizing outcomes.


In [1]:
import subprocess, re, csv, os, time
import torch
import pandas as pd
import matplotlib.pyplot as plt

c:\Users\jmigu\anaconda3\envs\tscil\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step 2: CUDA Compatibility and Execution Mode

The experiments were executed on a laptop equipped with an **Intel® Core™ Ultra 9 275HX (2.70 GHz)** processor  
and an **NVIDIA GeForce RTX 5070 Ti Laptop GPU** running on **CUDA 12.9** drivers.  

However, the **PyTorch version (1.13.1 + cu117)** required by the **TSCIL** framework supports GPU architectures only up to  
**compute capability sm_86** (e.g., RTX 30-series). The RTX 50-series GPUs rely on **sm_120**, which is recognized only by  
newer **PyTorch 2.5+** builds compiled with **CUDA 12.x**.  

Upgrading PyTorch would break compatibility with key dependencies such as **`fastai`** and **`tsai`**,  
which are restricted to older CUDA and PyTorch versions. Therefore, GPU acceleration cannot be used in this setup.

All experiments were consequently executed on the **CPU (Intel Ultra 9 275HX)** to ensure full compatibility, stability,  
and reproducibility across environments.

> 🧠 *Note:* Although CUDA drivers are correctly installed and detected,  
> PyTorch automatically falls back to CPU execution when the GPU’s compute capability is unsupported.


In [2]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

PyTorch version: 1.13.1+cu117
CUDA available: True
GPU: NVIDIA GeForce RTX 5070 Ti Laptop GPU


c:\Users\jmigu\anaconda3\envs\tscil\lib\site-packages\torch\cuda\__init__.py:155: UserWarning: 
NVIDIA GeForce RTX 5070 Ti Laptop GPU with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 compute_37.
If you want to use the NVIDIA GeForce RTX 5070 Ti Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


### Step 3: Experiment Pipeline Setup

To make our experimentation process **reproducible and systematic**, we define a reusable function called `run_experiment()`.

This function automates:
- The execution of the training script (`main_config.py`) with configurable arguments.  
- Live logging of training progress directly inside the notebook.  
- Extraction of key metrics such as:
  - **Average End Accuracy (Avg_End_Acc)** — final test accuracy over all tasks.  
  - **Average End Forgetting (Avg_End_Fgt)** — mean performance drop on previous tasks.  
  - **Average Current Accuracy (Avg_Cur_Acc)** — accuracy on the current task.  
- Saving of results to a CSV file (`results_log.csv`) for later comparison and analysis.

Each experiment is identified by its configuration parameters (agent type, encoder, normalization, memory size, etc.),  
making it easy to track and analyze multiple runs systematically.


In [5]:
import subprocess, re, csv, os, time

def run_experiment(agent="SFT", norm="LN", mem_budget=0.0, er_mode="task", 
                   epochs=20, encoder="CNN", data="har", seed=1234,
                   save_csv=True, results_file="results_log1.csv"):
    """
    Runs one experiment with live training logs (CPU-only), extracts key metrics,
    and saves results for later comparison.
    """
    # --- Force CPU execution ---
    env = os.environ.copy()
    env["CUDA_VISIBLE_DEVICES"] = ""  # disables CUDA globally for subprocess
    env["DEVICE"] = "cpu"

    cmd = [
        "python", "main_config.py",
        "--data", data,
        "--agent", agent,
        "--encoder", encoder,
        "--norm", norm,
        "--epochs", str(epochs),
        "--seed", str(seed),
        "--scenario", "class",
        "--debug", "True",
        "--device", "cpu",  # explicitly pass CPU device to script
    ]

    if agent == "ER":
        cmd += ["--mem_budget", str(mem_budget), "--er_mode", er_mode]
    
    print("\n🚀 Running (CPU-locked):", " ".join(cmd), "\n")
    start = time.time()

    process = subprocess.Popen(
        cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, env=env
    )

    log_lines = []
    for line in iter(process.stdout.readline, ''):
        if not line:
            break
        print(line, end='')  # live stream to notebook
        log_lines.append(line)
    
    process.wait()
    duration = time.time() - start
    log_text = "".join(log_lines)

    # --- Extract key metrics from log ---
    metrics = {}
    patterns = {
        "Avg_End_Acc": r"Avg_End_Acc\s*\[(.*?)\]",
        "Avg_End_Fgt": r"Avg_End_Fgt\s*\[(.*?)\]",
        "Avg_Cur_Acc": r"Avg_Cur_Acc\s*\[(.*?)\]",
    }
    for key, pat in patterns.items():
        match = re.search(pat, log_text)
        if match:
            try:
                metrics[key] = float(match.group(1))
            except:
                metrics[key] = None
        else:
            metrics[key] = None

    # --- Store result record ---
    record = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "agent": agent,
        "norm": norm,
        "encoder": encoder,
        "mem_budget": mem_budget,
        "er_mode": er_mode,
        "epochs": epochs,
        "seed": seed,
        "Avg_End_Acc": metrics["Avg_End_Acc"],
        "Avg_End_Fgt": metrics["Avg_End_Fgt"],
        "Avg_Cur_Acc": metrics["Avg_Cur_Acc"],
        "train_time_sec": round(duration, 2),
    }

    # --- Save to CSV file ---
    if save_csv:
        csv_exists = os.path.exists(results_file)
        with open(results_file, "a", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=list(record.keys()))
            if not csv_exists:
                writer.writeheader()
            writer.writerow(record)
        print(f"\n💾 Results saved to: {results_file}\n")

    print(f"✅ Finished {agent} ({norm}, mem={mem_budget}) on CPU in {duration:.1f}s")
    print(f"➡️ End Acc={metrics['Avg_End_Acc']}, Forget={metrics['Avg_End_Fgt']}, Cur Acc={metrics['Avg_Cur_Acc']}\n")

    return record


#### Experiment Configurations

We'll run the following experiments sequentially:

#### 🧪 Experiment Summary Table

| **ID** | **Agent** | **Encoder** | **Norm** | **Memory Budget** | **Epochs** | **Description / Purpose** |
|:------:|:-----------|:-------------|:----------|:------------------|:-----------|:---------------------------|
| **1** | SFT | CNN | LN | 0.00 | 20 | Baseline sequential fine-tuning. No replay or regularization. Demonstrates catastrophic forgetting. |
| **2** | ER | CNN | LN | 0.05 | 20 | Experience Replay with small buffer (5%). Tests effect of replay on stability and forgetting. |
| **3** | ER | CNN | BN | 0.05 | 20 | Same as Exp. 2 but with BatchNorm. Tests normalization impact on replay effectiveness. |
| **4** | EWC | CNN | LN | 0.00 | 20 | Elastic Weight Consolidation (regularization-based). Tests parameter-based continual learning. |
| **5a** | SFT | TST (Transformer) | LN | 0.00 | 20 | Sequential fine-tuning using Transformer encoder. Tests effect of architecture on forgetting. |
| **5b** | ER | TST (Transformer) | LN | 0.05 | 20 | Experience Replay with Transformer encoder. Tests synergy of replay with attention-based model. |
| **6** | ER | CNN | LN | 0.05 | 50 | Longer ER training. Tests convergence effects and replay stability over extended epochs. |
| **7** | ER | CNN | LN | 0.10 | 50 | Larger memory buffer (10%). Tests impact of increased replay memory on performance and forgetting. |


We begin by running the **baseline experiment** using the **Sequential Fine-Tuning (SFT)** strategy.  
In this setup, the model is trained sequentially on each task **without any replay buffer or regularization**,  
which allows us to clearly observe the phenomenon of **catastrophic forgetting**.

This baseline serves as the lower bound of continual learning performance —  
subsequent methods such as Experience Replay (ER) and EWC will be compared against it.

The following cell launches the baseline experiment using default parameters:
- **Agent:** `SFT`  
- **Encoder:** `CNN`  
- **Normalization:** `LayerNorm (LN)`  
- **Dataset:** `HAR`  
- **Epochs:** `20`


In [4]:
#res1 = run_experiment()

### 🧠 Experiment 2 — Experience Replay (ER)

In this experiment, we introduce **Experience Replay (ER)** — a memory-based continual learning strategy.  
It stores a subset of samples from previous tasks and replays them during training to reduce forgetting.

Configuration:
- Agent: `ER`
- Encoder: `CNN`
- Normalization: `LayerNorm (LN)`
- Memory Budget: `0.05`
- Epochs: `20`


In [ ]:
res2 = run_experiment(
    agent="ER",
    norm="LN",
    encoder="CNN",
    mem_budget=0.05,      
    er_mode="task",
    epochs=20,
    seed=0             
)



🚀 Running (CPU-locked): python main_config.py --data har --agent ER --encoder CNN --norm LN --epochs 20 --seed 0 --scenario class --debug True --device cpu --mem_budget 0.05 --er_mode task 

c:\Users\jmigu\anaconda3\envs\tscil\lib\site-packages\torch\cuda\__init__.py:155: UserWarning: 
NVIDIA GeForce RTX 5070 Ti Laptop GPU with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 compute_37.
If you want to use the NVIDIA GeForce RTX 5070 Ti Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
Namespace(agent='ER', scenario='class', stream_split='exp', data='har', encoder='CNN', head='Linear', criterion='CE', ncm_classifier=False, norm='LN', input_norm='IN', runs=1, epochs=20, batch_size=32, lr=0.001, 

### ⚙️ Experiment 3 — ER with Batch Normalization

This experiment tests whether **BatchNorm (BN)** performs as well as **LayerNorm (LN)**  
in non-stationary continual learning scenarios.  
Since BatchNorm depends on running statistics, it may be less stable when task distributions change.

Configuration:
- Agent: `ER`
- Encoder: `CNN`
- Normalization: `BatchNorm (BN)`
- Memory Budget: `0.05`
- Epochs: `20`


In [ ]:
res3 = run_experiment(
    agent="ER",
    norm="BN",             # Using BatchNorm instead of LayerNorm
    encoder="CNN",
    mem_budget=0.05,      
    epochs=20,
    seed=0
)



🚀 Running (CPU-locked): python main_config.py --data har --agent ER --encoder CNN --norm BN --epochs 20 --seed 0 --scenario class --debug True --device cpu --mem_budget 0.05 --er_mode task 

c:\Users\jmigu\anaconda3\envs\tscil\lib\site-packages\torch\cuda\__init__.py:155: UserWarning: 
NVIDIA GeForce RTX 5070 Ti Laptop GPU with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 compute_37.
If you want to use the NVIDIA GeForce RTX 5070 Ti Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
Namespace(agent='ER', scenario='class', stream_split='exp', data='har', encoder='CNN', head='Linear', criterion='CE', ncm_classifier=False, norm='BN', input_norm='IN', runs=1, epochs=20, batch_size=32, lr=0.001, 

### 🧩 Experiment 4 — Elastic Weight Consolidation (EWC)

Here, we test a **regularization-based approach** to continual learning:  
**Elastic Weight Consolidation (EWC)** penalizes large changes to important network weights,  
allowing the model to retain previously learned knowledge without using replay memory.

Configuration:
- Agent: `EWC`
- Encoder: `CNN`
- Normalization: `LayerNorm (LN)`
- Epochs: `20`


In [ ]:
res4 = run_experiment(
    agent="EWC",
    norm="LN",
    encoder="CNN",
    epochs=20,
    seed=0
)



🚀 Running (CPU-locked): python main_config.py --data har --agent EWC --encoder CNN --norm LN --epochs 20 --seed 0 --scenario class --debug True --device cpu 

c:\Users\jmigu\anaconda3\envs\tscil\lib\site-packages\torch\cuda\__init__.py:155: UserWarning: 
NVIDIA GeForce RTX 5070 Ti Laptop GPU with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 compute_37.
If you want to use the NVIDIA GeForce RTX 5070 Ti Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
Namespace(agent='EWC', scenario='class', stream_split='exp', data='har', encoder='CNN', head='Linear', criterion='CE', ncm_classifier=False, norm='LN', input_norm='IN', runs=1, epochs=20, batch_size=32, lr=0.001, lradj='step15', early_stop=True

### 🔍 Experiment 5 — Transformer Encoder (TST)

In this experiment, we replace the CNN encoder with a **Temporal Transformer (TST)**  
to evaluate how transformer-based architectures handle sequential time-series updates.

Configuration:
- Agent: `SFT`
- Encoder: `TST`
- Normalization: `LayerNorm (LN)`
- Epochs: `20`


In [ ]:
res5a = run_experiment(
    agent="SFT",
    norm="LN",
    encoder="TST",             
    epochs=20,
    seed=0
)



🚀 Running (CPU-locked): python main_config.py --data har --agent SFT --encoder TST --norm LN --epochs 20 --seed 0 --scenario class --debug True --device cpu 

c:\Users\jmigu\anaconda3\envs\tscil\lib\site-packages\torch\cuda\__init__.py:155: UserWarning: 
NVIDIA GeForce RTX 5070 Ti Laptop GPU with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 compute_37.
If you want to use the NVIDIA GeForce RTX 5070 Ti Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
Namespace(agent='SFT', scenario='class', stream_split='exp', data='har', encoder='TST', head='Linear', criterion='CE', ncm_classifier=False, norm='LN', input_norm='IN', runs=1, epochs=20, batch_size=32, lr=0.001, lradj='step15', early_stop=True

In [ ]:
res5b = run_experiment(
    agent="ER",
    norm="LN",
    encoder="TST",
    mem_budget=0.05, 
    er_mode="task",
    epochs=20,         
    seed=0
)



🚀 Running (CPU-locked): python main_config.py --data har --agent ER --encoder TST --norm LN --epochs 20 --seed 0 --scenario class --debug True --device cpu --mem_budget 0.05 --er_mode task 

c:\Users\jmigu\anaconda3\envs\tscil\lib\site-packages\torch\cuda\__init__.py:155: UserWarning: 
NVIDIA GeForce RTX 5070 Ti Laptop GPU with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 compute_37.
If you want to use the NVIDIA GeForce RTX 5070 Ti Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
Namespace(agent='ER', scenario='class', stream_split='exp', data='har', encoder='TST', head='Linear', criterion='CE', ncm_classifier=False, norm='LN', input_norm='IN', runs=1, epochs=20, batch_size=32, lr=0.001, 

### 🔁 Experiment 6 — Longer ER Training (50 Epochs)

Finally, we extend the **Experience Replay (ER)** training duration to assess stability over longer runs.  
This experiment helps identify whether additional training improves consolidation or leads to overfitting.

Configuration:
- Agent: `ER`
- Encoder: `CNN`
- Normalization: `LayerNorm (LN)`
- Memory Budget: `0.05`
- Epochs: `50`


In [ ]:
res6 = run_experiment(
    agent="ER",
    norm="LN",
    encoder="CNN",
    mem_budget=0.05,         
    er_mode="task",          
    epochs=50,                # extended training for better convergence
    seed=0
)



🚀 Running (CPU-locked): python main_config.py --data har --agent ER --encoder CNN --norm LN --epochs 50 --seed 0 --scenario class --debug True --device cpu --mem_budget 0.05 --er_mode task 

c:\Users\jmigu\anaconda3\envs\tscil\lib\site-packages\torch\cuda\__init__.py:155: UserWarning: 
NVIDIA GeForce RTX 5070 Ti Laptop GPU with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 compute_37.
If you want to use the NVIDIA GeForce RTX 5070 Ti Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
Namespace(agent='ER', scenario='class', stream_split='exp', data='har', encoder='CNN', head='Linear', criterion='CE', ncm_classifier=False, norm='LN', input_norm='IN', runs=1, epochs=50, batch_size=32, lr=0.001, 

### 🔁 Experiment 7 - Larger Memory Buffer

In this experiment, we increase the memory buffer for Experience Replay (ER) to 10%.  
This aims to assess the impact of a larger memory buffer on the model's ability to retain knowledge and mitigate forgetting.

In [4]:
res7 = run_experiment(
    agent="ER",
    norm="LN",
    encoder="CNN",
    mem_budget=0.10,         
    er_mode="task",          
    epochs=50,                
    seed=0
)



🚀 Running (CPU-locked): python main_config.py --data har --agent ER --encoder CNN --norm LN --epochs 50 --seed 0 --scenario class --debug True --device cpu --mem_budget 0.1 --er_mode task 

c:\Users\jmigu\anaconda3\envs\tscil\lib\site-packages\torch\cuda\__init__.py:155: UserWarning: 
NVIDIA GeForce RTX 5070 Ti Laptop GPU with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 compute_37.
If you want to use the NVIDIA GeForce RTX 5070 Ti Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
Namespace(agent='ER', scenario='class', stream_split='exp', data='har', encoder='CNN', head='Linear', criterion='CE', ncm_classifier=False, norm='LN', input_norm='IN', runs=1, epochs=50, batch_size=32, lr=0.001, l